In [1]:
# necessary imports
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score

np.random.seed(19)

In [2]:
data_folder = ""
data = pd.read_csv(os.path.join(data_folder, "mushrooms.csv"))

In [3]:
data

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [5]:
data["class"] = data["class"].apply(lambda row: -1 if row[0] == 'e' else 1)

In [6]:
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,-1,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,-1,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,1,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,-1,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [17]:
def dummies(data, columns = ["pclass", "name_title", "embarked", "sex"]):
    for col in columns:
        data[col] = data[col].apply(lambda x: str(x))
        new_cols = [col + "_" + i for i in data[col].unique()]
        data = pd.concat([data, pd.get_dummies(data[col], prefix=col)[new_cols]], axis = 1)
        del data[col]
    return data

In [18]:
target = "class"
cols = data.columns.drop(target)

In [19]:
cols

Index(['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [20]:
data_set = dummies(data, columns = cols)

In [21]:
data_set.head()

,class,cap-shape_x,cap-shape_b,cap-shape_s,cap-shape_f,cap-shape_k,cap-shape_c,cap-surface_s,cap-surface_y,cap-surface_f,...,population_v,population_y,population_c,habitat_u,habitat_g,habitat_m,habitat_d,habitat_p,habitat_w,habitat_l
0,1,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,-1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,-1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,-1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [22]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_set, test_size = 0.3)

In [24]:
class TreeNode:
    def __init__(self, is_leaf, prediction, split_feature):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.split_feature = split_feature
        self.left = None
        self.right = None

In [23]:
# check which classification gives the smallest weighted mistake
# return weight and classification
def node_weighted_mistakes(targets_in_node, data_weights):
    weight_positive = sum(data_weights[targets_in_node == 1])
    weighted_mistakes_negative = weight_positive
    weight_negative = sum(data_weights[targets_in_node == -1])
    weighted_mistakes_positive = weight_negative
    
    if weighted_mistakes_negative < weighted_mistakes_positive:
        return (weighted_mistakes_negative, -1)
    else:
        return (weighted_mistakes_positive, 1)

In [25]:
example_targets = np.array([-1, -1, 1, 1, 1])
example_data_weights = np.array([1., 2., 0.5, 1., 1.])
node_weighted_mistakes(example_targets, example_data_weights)

(2.5, -1)

In [26]:
def best_split_weighted(data, features, target, data_weights):
    best_features = None
    best_error = float("inf")
    num_data_points = float(len(data))
    
    for feature in features:
        left_split = data[data[feature] == 0]
        left_data_weights = data_weights[data[feature] == 0]
        
        right_split = data[data[feature] == 1]
        right_data_weights = data_weights[data[feature] == 1]
        
        left_misses, left_class = node_weighted_mistakes(left_split[target], left_data_weights)
        right_misses, right_class = node_weighted_mistakes(right_split[target], right_data_weights)
        
        error = (left_misses + right_misses) * 1.0 / sum(data_weights)
        
        if error < best_error:
            best_error = error
            best_feature = feature
    return best_feature

In [27]:
features = data_set.columns.drop(target)
example_data_weights = np.array(len(train_data) * [2])
best_split_weighted(train_data, features, target, example_data_weights)

'odor_n'

In [28]:
def create_leaf(target_values, data_weights):
    leaf = TreeNode(True, None, None)
    weighted_error, prediction_class = node_weighted_mistakes(target_values, data_weights)
    leaf.prediction = prediction_class
    return leaf

In [37]:
def create_weighted_tree(data, data_weights, features, target, current_depth = 0, max_depth = 10, min_error = 1e-15):
    remaining_features = features[:]
    target_values = data[target]
    
    if node_weighted_mistakes(target_values, data_weights)[0] <= min_error:
        print("Termination 1 reached")
        return create_leaf(target_values, data_weights)
    if len(remaining_features) == 0:
        print("Termination 2 reached")
        return create_leaf(target_values, data_weights)
    if current_depth >= max_depth:
        print("Termination 3 reached")
        return create_leaf(target_values, data_weights)
    
    split_feature = best_split_weighted(data, features, target, data_weights)
    print(f"split on feature {split_feature}")
    
    left_split = data[data[split_feature] == 0]
    right_split = data[data[split_feature] == 1]
    
    left_data_weights = data_weights[data[split_feature] == 0]
    right_data_weights = data_weights[data[split_feature] == 1]
    
    remaining_features = remaining_features.drop(split_feature)
    
    if len(left_split) == len(data):
        return create_leaf(left_split[target], left_data_weights)
    if len(right_split) == len(data):
        return create_leaf(right_split[target], right_data_weights)
    
    left_tree = create_weighted_tree(left_split, left_data_weights, remaining_features, target, current_depth + 1, max_depth, min_error)
    right_tree = create_weighted_tree(right_split, right_data_weights, remaining_features, target, current_depth + 1, max_depth, min_error)
    
    result_node = TreeNode(False, None, split_feature)
    result_node.left = left_tree
    result_node.right = right_tree
    
    return result_node

In [38]:
def count_leaves(tree):
    if tree.is_leaf:
        return 1
    return count_leaves(tree.left) + count_leaves(tree.right)

In [41]:
example_data_weights = np.array([1.0 for i in range(len(train_data))])
small_data_decision_tree = create_weighted_tree(train_data, example_data_weights, features, target, max_depth = 2, min_error = 1e-15)
count_leaves(small_data_decision_tree)

split on feature odor_n
split on feature stalk-root_c
Termination 3 reached
Termination 3 reached
split on feature spore-print-color_r
Termination 3 reached
Termination 1 reached


4

In [42]:
def predict_single_data(tree, x, annotate = False):
    if tree.is_leaf:
        if annotate:
            print("leaf node, predicting %s" % tree.prediction)
        return tree.prediction
    else:
        split_feature_value = x[tree.split_feature]
        
        if annotate:
            print("split on %s = %s" % (tree.split_feature, split_feature_value))
        if split_feature_value == 0:
            return predict_single_data(tree.left, x, annotate)
        else:
            return predict_single_data(tree.right, x, annotate)

In [45]:
def evaluate_accuracy(tree, data):
    prediction = data.apply(lambda row: predict_single_data(tree, row), axis = 1)
    accuracy = (prediction == data[target]).sum() * 1.0 / len(data)
    return accuracy

In [46]:
evaluate_accuracy(small_data_decision_tree, test_data)

0.9524200164068909

In [47]:
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
class WeightedDecisionTree(BaseEstimator):
    def __init__(self, max_depth, min_error):
        self.max_depth = max_depth
        self.min_error = min_error
        
    def fit(self, X, Y, data_weights = None):
        data_set = pd.concat([X, Y], axis = 1)
        features = X.columns
        target = Y.columns[0]
        self.root_node = create_weighted_tree(data_set, data_weights, features, target,
                                             current_depth = 0, max_depth = self.max_depth, min_error = self.min_error)
        
    def predict(self, X):
        prediction = X.apply(lambda row: predict_single_data(self.root_node, row), axis = 1)
        return prediction
    
    def score(self, testX, testY):
        target = testY.columns[0]
        result = self.predict(testX)
        return accuracy_score(testY[target], result)

In [55]:
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
class MyAdaBoost(BaseEstimator):
    def __init__(self, M):
        self.M = M
    
    def fit(self, X, Y):
        self.models = []
        self.model_weights = []
        self.target = Y.columns[0]
        
        N, _ = X.shape
        alpha = np.ones(N) / N
        
        for m in range(self.M):
            tree = WeightedDecisionTree(max_depth = 2, min_error = 1e-15)
            tree.fit(X, Y, data_weights = alpha)
            prediction = tree.predict(X)
            
            weighted_error = alpha.dot(prediction != Y[self.target])
            model_weight = 0.5 * (np.log(1-weighted_error) - np.log(weighted_error))
            alpha = alpha * np.exp(-model_weight * Y[self.target] * prediction)
            alpha = alpha / alpha.sum()
            
            self.models.append(tree)
            self.model_weights.append(model_weight)
            
    def predict(self, X):
        N, _ = X.shape
        result = np.zeros(N)
        for wt, tree in zip(self.model_weights, self.models):
            result += wt * tree.predict(X)
        return np.sign(result)
    
    def score(self, testX, testY):
        result = self.predict(testX)
        return accuracy_score(testY[self.target], result)

In [56]:
m = MyAdaBoost(20)

In [57]:
trainX = train_data[features]
trainY = train_data[[target]]
m.fit(trainX, trainY)

split on feature odor_n
split on feature stalk-root_c
Termination 3 reached
Termination 3 reached
split on feature spore-print-color_r
Termination 3 reached
Termination 1 reached
split on feature ring-type_p
split on feature stalk-root_e
Termination 3 reached
Termination 1 reached
split on feature odor_f
Termination 3 reached
Termination 1 reached
split on feature gill-size_n
split on feature stalk-surface-above-ring_k
Termination 3 reached
Termination 3 reached
split on feature odor_a
Termination 3 reached
Termination 1 reached
split on feature spore-print-color_r
split on feature odor_p
Termination 3 reached
Termination 1 reached
Termination 1 reached
split on feature odor_l
split on feature ring-type_f
Termination 3 reached
Termination 1 reached
Termination 1 reached
split on feature odor_f
split on feature odor_c
Termination 3 reached
Termination 1 reached
Termination 1 reached
split on feature cap-surface_f
split on feature habitat_w
Termination 3 reached
Termination 1 reached
spl

In [58]:
testX = test_data[features]
testY = test_data[[target]]
m.score(testX, testY)

1.0